In [ ]:
#iris dataset classification example
import pandas as pd
colnames = ['sepal_length','sepal_width','petal_length','petal_width','class']
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=colnames)
data


In [ ]:
data['setosa'] = data['class'] == 'Iris-setosa'
data['setosa']


In [ ]:
# convert to streaming dataset
for i,row in data.sample(1).iterrows():
    X = row[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
    X = X.to_dict()
    
    y = row['setosa']
    
    print(X)
    print(y)
    break


In [ ]:
!pip install river
from river import linear_model

model =  linear_model.LogisticRegression()
for i,row in data.sample(frac=1).iterrows():
    X = row[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
    X = X.to_dict()
    y = row['setosa']
        
    model.learn_one(X, y)


In [ ]:
preds = model.predict_many(data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
print(preds)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(data['setosa'], preds)


In [ ]:
# add a stratified train test split
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, stratify =data['setosa'])


In [ ]:
from river import linear_model,metrics

model =  linear_model.LogisticRegression()
correct = []

for i,row in train.sample(frac=1).iterrows():
    X = row[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
    X = X.to_dict()
    
    y = row['setosa']
    
    model.predict_one(X)
    correct.append(y == model.predict_one(X))
    
    model.learn_one(X,y)


In [ ]:
# this model is learning quite stable from the start

import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.cumsum(correct))


In [ ]:
# model was not so good on out of sample
accuracy_score(test['setosa'],model.predict_many(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]))


In [ ]:
# add a stratified train test split
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, stratify =data['class'])


In [ ]:
from river import linear_model,metrics,multiclass
model =  multiclass.OneVsRestClassifier(linear_model.LogisticRegression())
correct = []

for i,row in train.sample(frac=1).iterrows():
    X = row[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
    X = X.to_dict()
    
    y = row['class']
    
    model.predict_one(X)
    correct.append(y == model.predict_one(X))

    model.learn_one(X,y)


In [ ]:
# this model predicts better after 40 observations

import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.cumsum(correct))


In [ ]:
model.predict_many(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [ ]:
# model scores 0.63 on the test data
from sklearn.metrics import accuracy_score
accuracy_score(test['class'],model.predict_many(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]))
